### Algoritmo NSGAII

O Algoritmo NSGAII trabalha como um operador de seleção de indivíduos sobre uma população. É mais rápido e possui uma menor complexidade computacional se comparado com outros algoritmos notórios. A adoção do método elitista garante uma maior velocidade de sua execução.

Inicialmente uma população P(t) de tamanho N (t=0) é criada aleatoriamente e ordenada de acordo com sua não-dominância. Para cada solução é atribuído um valor de fitness (ou rank) equivalente ao seu nível de não-dominância, sendo o nível 1 o melhor, nível 2 o melhor seguinte e etc.

Em seguida Q(t), de tamanho N e t=0, é gerado utilizando operadores de seleção de torneiro binário, recombinação e mutação.

Feito isso, combina-se P(t) e Q(t) (pais e filhos) obtendo-se R(t) de tamanho 2N. O elitismo é garantido neste processo pois, até o momento, todos os indivíduos gerados estão preservados. 

As soluções de R(t) são ordenadas de acordo com sua não-dominância, formando as fronteiras, desde F1 (fronteira que contém as melhores soluções) até a última identificada. O pior caso ocorre quando para cada fronteira há apenas uma solução.

O objetivo seguinte é obter a próxima geração P(t+1) de tamanho N a partir de R(t). Logo, se o tamanho da fronteira F1 de R(t) for menor que N todas as soluções de F1 passarão a pertencer a P(t+1). E soluções das fronteiras seguintes são adicionadas a P(t+1) até que seu tamanho seja N, de acordo com o ranking de não dominância.

<div align='center'><img src='https://www.researchgate.net/profile/N_Nariman-Zadeh/publication/228569970/figure/fig1/AS:302014627106822@1449017306661/Fig-1-Basics-of-NSGA-II-procedure.png' width='56%' align='center'/>
Fonte: https://www.researchgate.net/publication/228569970_Thermodynamic_Pareto_optimization_of_turbojet_engines_using_multi-objective_genetic_algorithms</div>

Durante a geração de P(t+1) calcula-se para cada indivíduo seu ranking e distância de agrupamento, necessários na hora de comparar as soluções e obter a melhor através do operador crowded-comparison.

Agora, sobre a nova população P(t+1) de tamanho N utilizamos operadores de seleção, mutação e cruzamento para criar a nova população Q(t+1) de tamanho N. 

Este processo se repete até que um critério de parada seja atingido. A função NSGA2 é composta por outras funções auxiliares, conforme descrito a seguir.


In [1]:
#função responsável por determinar a dominância de um indivíduo sobre o outro.
#ind1 dominará ind2 se:
#    - ind1 e ind2 forem diferentes entre si e 
#    - ind1 preceder ind2
def dominates(ind1, ind2):
    
    #verifica igualdade
    r1 = np.array_equal(ind1, ind2)
    if r1 == True:
        return False
    
    #verifica a precedência  
    for i in range(len(ind1)):               
        if ind1[i] > ind2[i]:
            return False       
        
    return True

In [72]:
#função que identifica as soluções de acordo com cada fronteira existente, classificando-os segundo
#    seu nível de não dominância
def fast_non_dominated_sort(P):
    
    #início do loop que analisa cada indivíduo p da população P
    #este primeiro laço é responsável por identificar os indivíduos da primeira fronteira
    for p in P:
    
        
        n_p = 0 #contador de dominância, quantidade de soluções que p domina, todas as soluções da primeira
            #fronteira possuem n_p = 0
        S_p = [] #lista das soluções dominadas por p
        F = {} #lista(dicionário) das fronteiras
        F[1] = []
        
        #compara a solução p com as demais soluções da população P, determinando sua relação de dominância
        for q in P:
    
            if dominates(p, q): #se p domina q, q é adicionado ao conjunto de soluções dominadas por p
                S_p.append(q) 
                #p.dominated_solutions.append(q)
            elif dominates(q, p): #senão o contador de dominância de p é incrementado
                n_p = n_p + 1
          
        #se o contador de dominância do p em questão for igual a zero ele pertencerá à primeira fronteira
        if n_p == 0:
            p.rank = 1 #soluções da fronteira 1 possuem rank igual a 1
            #F.append(p) 
            F[1].append(p) 
        
        p.dominated_solutions = S_p
        p.domination_counter = n_p
        
    
    #este segundo laço é responsável por identificar os indivíduos das demais fronteiras
    i = 1
    for x in F[i]:
        Q = [] #armazena os indivíduos da próxima fronteira     
        
        for p in F[i]:
            #for q in S_p:
            for q in p.dominated_solutions:
                n_q = p.domination_counter - 1
                
                if n_q == 0:
                    q.rank = i + 1
                    Q.append(q)
                    
        i = i + 1
        F[i] = Q         
        
    return F

In [73]:
def crowding_distance_assignmnet(I):
    
    l = len(I)
    individuals = []

    for i in I:
        individuals.append((i,0));

        
        individuals.sort()

    return individuals

In [74]:
def crowded_comparison_operator(ind1, ind2):
    
    if (ind1.rank < ind2.rank) or ((ind1.rank == ind2.rank) and (ind1.crowding_distance > ind2.crowding_distance)):
        return True
    else:
        return False

In [75]:
def rodrigoNSGA2(pop,tam):
    
    #ordeno os indivíduos da população de acordo com sua não-dominância
    #F = []
    F = fast_non_dominated_sort(pop)
    
    i=1
    P = []
    while len(P)+len(F) <= N:
        crowding_distance_assignmnet(F)
        P = P.append(F)
        i = i + 1
        
    #classifica as soluções em ordem decrescente usando o operador crowded_comparison
    crowded_comparison_operator(F)
    
    P = P.append(F[1:(N-len(P))])
    Q = make_new_pop(P)
    
    t = t + 1    
    

In [76]:
import time, array, random, copy, math
import numpy as np
import pandas as pd

from pprint import pprint

In [77]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] ='\\usepackage{libertine}\n\\usepackage[utf8]{inputenc}'

import seaborn
seaborn.set(style='whitegrid')
seaborn.set_context('notebook')

In [78]:
from deap import algorithms, base, benchmarks, tools, creator

In [79]:
random.seed(a=42)

In [80]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0))
#creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMin)
#creator.create("Individual", list, typecode='d', fitness=creator.FitnessMin)
#creator.create("Individual", array.array, 
creator.create("Individual", list, 
                #typecode='d', 
                fitness=creator.FitnessMin,
                rank = None,
                crowding_distance = None,
                #dominated_solutions = set(),
                dominated_solutions = [],
                features = None,
                objectives = None,
                #dominates = None,
                domination_counter = 0,
              )

In [81]:
def uniform(low, up, size=None):
    try:
        return [random.uniform(a, b) for a, b in zip(low, up)]
    except TypeError:
        return [random.uniform(a, b) for a, b in zip([low] * size, [up] * size)]

In [82]:
toolbox = base.Toolbox()

In [83]:
#NDIM = 30 
NDIM = 3 
BOUND_LOW, BOUND_UP = 0.0, 1.0
toolbox.register("evaluate", lambda ind: benchmarks.dtlz3(ind, 2))

In [84]:
toolbox.register("attr_float", uniform, BOUND_LOW, BOUND_UP, NDIM)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0)
toolbox.register("mutate", tools.mutPolynomialBounded, low=BOUND_LOW, up=BOUND_UP, eta=20.0, indpb=1.0/NDIM)
#toolbox.register("select", tools.selNSGA2)
toolbox.register("select", rodrigoNSGA2)

In [85]:
a=creator.Individual([1,2,3])
a.rank=1
#a=[1,2,3]
pop=[[1,2,3],[3,4,6],[43,42,63],[13,34,36],[23,74,96],[23,54,86],[23,64,68],[43,49,69],[83,64,86],[93,64,96]]
pop

[[1, 2, 3],
 [3, 4, 6],
 [43, 42, 63],
 [13, 34, 36],
 [23, 74, 96],
 [23, 54, 86],
 [23, 64, 68],
 [43, 49, 69],
 [83, 64, 86],
 [93, 64, 96]]

In [86]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
a=toolbox.population(5)
#a[0].rank = 2

In [87]:
from pprint import pprint

pprint(vars(a))

TypeError: vars() argument must have __dict__ attribute

In [88]:
from random import randint
#print(randint(0, 9))
for i in a:
    i.rank = randint(0, 9)
    print(i, " rank= ", i.rank)

[0.6394267984578837, 0.025010755222666936, 0.27502931836911926]  rank=  8
[0.22321073814882275, 0.7364712141640124, 0.6766994874229113]  rank=  6
[0.8921795677048454, 0.08693883262941615, 0.4219218196852704]  rank=  3
[0.029797219438070344, 0.21863797480360336, 0.5053552881033624]  rank=  7
[0.026535969683863625, 0.1988376506866485, 0.6498844377795232]  rank=  9


In [89]:
a

[[0.6394267984578837, 0.025010755222666936, 0.27502931836911926],
 [0.22321073814882275, 0.7364712141640124, 0.6766994874229113],
 [0.8921795677048454, 0.08693883262941615, 0.4219218196852704],
 [0.029797219438070344, 0.21863797480360336, 0.5053552881033624],
 [0.026535969683863625, 0.1988376506866485, 0.6498844377795232]]

In [90]:
f=fast_non_dominated_sort(a)

In [91]:
f

{1: [[0.026535969683863625, 0.1988376506866485, 0.6498844377795232]], 2: []}

In [28]:
a = sorted(a, key=lambda a: a.rank)   # sort by age

In [29]:
a

[[0.026535969683863625, 0.1988376506866485, 0.6498844377795232],
 [0.6981393949882269, 0.3402505165179919, 0.15547949981178155],
 [0.9572130722067812, 0.33659454511262676, 0.09274584338014791],
 [0.5449414806032167, 0.2204406220406967, 0.5892656838759087],
 [0.8094304566778266, 0.006498759678061017, 0.8058192518328079]]

In [ ]:
toolbox.pop_size = 50
toolbox.max_gen = 1000
toolbox.mut_prob = 0.2

In [ ]:
#p = toolbox.population(n=toolbox.pop_size)

In [ ]:
#pprint(vars(p[0]))

In [ ]:
#toolbox.select(p[0], len(p[0]))

In [ ]:
def run_ea(toolbox, stats=None, verbose=False):
    pop = toolbox.population(n=toolbox.pop_size)
    pop = toolbox.select(pop, len(pop))   
    return algorithms.eaMuPlusLambda(pop, 
                                     toolbox, 
                                     mu=toolbox.pop_size, 
                                     lambda_=toolbox.pop_size, 
                                     cxpb=1-toolbox.mut_prob,
                                     mutpb=toolbox.mut_prob, 
                                     stats=stats, 
                                     ngen=toolbox.max_gen, 
                                     verbose=verbose)

In [ ]:
run_ea(toolbox)

In [ ]:
%time res,_ = run_ea(toolbox)

In [ ]:
fronts = tools.emo.sortLogNondominated(res, len(res))

In [ ]:
plot_colors = seaborn.color_palette("Set1", n_colors=10)
fig, ax = plt.subplots(1, figsize=(4,4))
for i,inds in enumerate(fronts):
    par = [toolbox.evaluate(ind) for ind in inds]
    df = pd.DataFrame(par)
    df.plot(ax=ax, kind='scatter', label='Front ' + str(i+1), 
                 x=df.columns[0], y=df.columns[1], 
                 color=plot_colors[i])
plt.xlabel('$f_1(\mathbf{x})$');plt.ylabel('$f_2(\mathbf{x})$');